In [28]:
import pandas as pd
import numpy as np

In [29]:
df= pd.read_csv('./data/LoanStats3a.csv',skiprows=1, low_memory=True)
#df.head()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Columns: 145 entries, id to settlement_term
dtypes: float64(115), object(30)
memory usage: 47.1+ MB
None


C:\Users\Administrator\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df.drop('id',axis = 1, inplace = True)#删除无用数据
df.drop('member_id',axis = 1,inplace = True)
df.drop('sub_grade', axis = 1, inplace = True)#次级等级
df.drop('emp_title', axis = 1, inplace = True)#emp_title借款人申请贷款时提供的职称
# 36 months 将 单位去掉  只剩下数字
df['term'].replace(to_replace = '[^0-9]+', value = '',inplace = True,regex = True)#贷款时长

In [31]:
df['int_rate'].replace(to_replace = '%', value = '',inplace = True,regex = True)#贷款利率
df['revol_util'].replace(to_replace = '%', value = '',inplace = True,regex = True)
df.emp_length.replace('n/a',np.nan,inplace = True)
df.emp_length.replace(to_replace = '[^0-9]+',value='',inplace = True,regex=True)#工作经验
df.dropna(axis = 1, how = 'all', inplace = True)
df.dropna(axis = 0, how = 'all', inplace = True)
#删除空值较多的列
df.drop(['debt_settlement_flag_date','settlement_status','settlement_date',\
         'settlement_amount','settlement_percentage',\
         'settlement_term'], axis = 1, inplace = True)
#删除样本重复较多的值
df.drop(['delinq_2yrs','inq_last_6mths','mths_since_last_delinq',\
         'mths_since_last_record','open_acc','pub_rec','total_acc',\
         'out_prncp','out_prncp_inv','collections_12_mths_ex_med',\
         'policy_code','acc_now_delinq','chargeoff_within_12_mths',\
         'delinq_amnt','pub_rec_bankruptcies',\
         'tax_liens'], axis = 1, inplace = True)

df.drop(['desc','title','term','grade','emp_length','home_ownership','verification_status'\
         ,'issue_d','pymnt_plan','purpose','zip_code','addr_state',\
         'earliest_cr_line','initial_list_status','last_pymnt_d',\
         'next_pymnt_d','last_credit_pull_d','application_type','hardship_flag',
         'disbursement_method','debt_settlement_flag'], axis = 1, inplace = True)
#标签二值化
df.loan_status.replace('Fully Paid',value=int(0),inplace = True)
df.loan_status.replace('Charged Off',value=int(1),inplace = True)
df.loan_status.replace('Does not meet the credit policy. Status:Fully Paid', \
                       np.nan, inplace = True)
df.loan_status.replace('Does not meet the credit policy. Status:Charged Off', \
                       np.nan, inplace = True)

In [32]:
df.loan_status.value_counts()
df.dropna(subset=['loan_status'],how = 'any',inplace = True)
df.fillna(0.0, inplace = True)#填充

In [33]:
df = df.convert_objects(convert_numeric=True)
df.head()

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,loan_status,dti,revol_bal,revol_util,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt
0,5000.0,5000.0,4975.0,10.65,162.87,24000.0,0.0,27.65,13648.0,83.7,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,171.62
1,2500.0,2500.0,2500.0,15.27,59.83,30000.0,1.0,1.00,1687.0,9.4,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,119.66
2,2400.0,2400.0,2400.0,15.96,84.33,12252.0,0.0,8.72,2956.0,98.5,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,649.91
3,10000.0,10000.0,10000.0,13.49,339.31,49200.0,0.0,20.00,5598.0,21.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,357.48
4,3000.0,3000.0,3000.0,12.69,67.79,80000.0,0.0,17.94,27783.0,53.9,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,67.30


In [34]:
#计算各个特征之间的相关性将大于0.95的特征删除
cor = df.corr()
cor.iloc[:, :] = np.tril(cor, k= -1)
cor = cor.stack()
#print(cor[(cor>0.55)|(cor<-0.55)])
df.drop(['loan_amnt','funded_amnt','total_pymnt','total_pymnt'], axis = 1, inplace = True)

In [35]:
df.describe()
df.to_csv('./data/feature03.csv')